# Dependencies

In [1]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!pip install indic-nlp-library

!pip install pytorch-pretrained-bert
# git clone model
!git clone https://github.com/imranulashrafi/banner.git
# copy model to models
!cp /kaggle/input/gpun-indic-banner-weights/models/18.pt banner/models/
# download dataset
!git clone https://github.com/MISabic/NER-Bangla-Dataset.git
# unzip 
!unzip NER-Bangla-Dataset/Bangla-NER-Splitted-Dataset.zip -d dataset 
# move model to models
!cp /kaggle/input/gpun-bnnorm-banner-weights/banner/models/banner_model.pt banner/models/banner_model.pt

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 35.87 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 

# Dataset

In [2]:
#-------------------------
# imports
#-------------------------
import sys
sys.path.append("./banner")
import json
import os
import pandas as pd 
from tqdm import tqdm

from sklearn.metrics import classification_report
from net import Net
from dataset import NerDataset, pad, VOCAB
from trainer import eval
import torch
import numpy as np
import random

tqdm.pandas()

# initialize
INDIC_NLP_RESOURCES="/kaggle/working/indic_nlp_resources/"
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
factory=IndicNormalizerFactory()

bnorm=factory.get_normalizer("bn")
#-------------------------
# format raw test set
#-------------------------
with open("./dataset/Bangla-NER-Splitted-Dataset.json","r") as f:
    dataset=json.load(f)



100%|██████████| 995526/995526 [00:00<00:00, 8226819.49B/s]


In [3]:

def infect(word, random_number = 75):
    nukta_map              =   {'য়':'য়',
                            'র':'ব়',
                            'ড়':'ড়',
                            'ঢ়':'ঢ়'}
    diacritic_map           =   {'ো':'ো',
                                'ৌ':'ৌ',
                                'অা':'আ',
                                'ৃ':'ৄ'}


    vowel_diacritics       =   ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ']

    non_gylph_unicodes     =   ['\u0984', '\u098d','\u098e','\u0991','\u0992','\u09a9','\u09b1','\u09b3','\u09b4','\u09b5',
                                    '\u09ba','\u09bb', '\u09c5','\u09c6','\u09c9','\u09ca','\u09cf','\u09d0','\u09d1','\u09d2',
                                    '\u09d3','\u09d4','\u09d5','\u09d6', '\u09d8','\u09d9','\u09da','\u09db','\u09de', '\u09e4',
                                    '\u09e5','\u09ff']

    vowels                 =   ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ']

    consonants             =   ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ','জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 
                                    'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ','ড়', 'ঢ়', 'য়','ৎ']


    connector              =   '্'
    nukta                  =   '়'
    original="".join([c for c in word])
    try:
    
        random_number = random_number % 5


        part = [i for i in word]
        new_word = []

        if random.randint(0, 100) > 70:
            index_connector = random.randint(1, len(word))
            word = word[:index_connector] + non_gylph_unicodes[random.randint(0, len(non_gylph_unicodes))] + word[index_connector:]
            index_connector = random.randint(1, len(word))
            word = word[:index_connector] + connector + word[index_connector:]

        for i in word:
            if i in vowels:
                if random.randint(0, 100) > 50:
                    word = word.replace(i, i+vowel_diacritics[random.randint(0, len(vowel_diacritics))])

        for i in part:
            for k in list(nukta_map):
                if i == k:
                    word = word.replace(k, nukta_map[k])

        for i in range(len(part)):
            for k in list(diacritic_map):
                if part[i] == k:
                    d_part = [_ for _ in diacritic_map[k]]
                    word = ("".join(part[:i]) + d_part[0] + d_part[1] + "".join(part[i+1:]))
                    #word = "".join(part)

        return word
    except Exception as e:
        return original


In [4]:
def norm_sentence(sent):
    words=[]
    for word in sent:
        word=bnorm.normalize(word)
        word=word.replace(" ","")
        words.append(word)
    return words

* original --> df
* normalized--> dfn
* infected1-->df1
* infected2-->df2
* infected5-->df5
* infected1 norm-->df1n
* infected2 norm-->df2n
* infected5 norm-->df5n


In [5]:
# original
df=pd.DataFrame(dataset["test"])
# normalied
dfn=df.copy() 
dfn.sentence=dfn.sentence.progress_apply(lambda x:norm_sentence(x))

# infected-1
df1=df.copy() 
df1.sentence=df1.sentence.progress_apply(lambda x:[infect(i) for i in x])

# infected-2
df2=df1.copy() 
df2.sentence=df2.sentence.progress_apply(lambda x:[infect(i) for i in x])
# infected-5
df5=df.copy()
for i in range(5):
    df5.sentence=df5.sentence.progress_apply(lambda x:[infect(i) for i in x])

# infected-1-norm
df1n=df1.copy() 
df1n.sentence=df1n.sentence.progress_apply(lambda x:norm_sentence(x))
# infected-2-norm
df2n=df2.copy() 
df2n.sentence=df2n.sentence.progress_apply(lambda x:norm_sentence(x))
# infected-5-norm
df5n=df5.copy() 
df5n.sentence=df5n.sentence.progress_apply(lambda x:norm_sentence(x))


100%|██████████| 3564/3564 [00:00<00:00, 8114.26it/s]


# Load models

In [6]:
# model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
top_rnns=True
model_s = Net(top_rnns, len(VOCAB), device, finetuning=True)
model_b = Net(top_rnns, len(VOCAB), device, finetuning=True)

model_s.load_state_dict(torch.load('banner/models/18.pt'))
model_s=model_s.to(device)
# zero - shot
model_b.load_state_dict(torch.load('banner/models/banner_model.pt'))
model_b=model_b.to(device)

def run_ner_infer(sent,model):
    tags = []
    for x in range(len(sent.split())):
        tags.append('O')
    sent_infer=[]
    sent_infer.append(["[CLS]"] + sent.split() + ["[SEP]"])
    tags_infer=[]
    tags_infer.append(["<PAD>"] + tags + ["<PAD>"])

    infer_data = NerDataset(sent_infer, tags_infer)

    infer_iter = torch.utils.data.DataLoader(dataset=infer_data,
                             batch_size=1,
                             shuffle=False,
                             collate_fn = pad,
                             num_workers=0
                             )
    pred = eval(model, infer_iter)
    for x in range(len(pred[0])):
        if pred[0][x] == '<PAD>':
            pred[0][x] = 'O'
    return pred[0][1:-1]

100%|██████████| 662804195/662804195 [00:12<00:00, 53782081.23B/s]


In [7]:
def eval_data(df):
    preds=[]
    gts=[]
    for idx in tqdm(range(len(df))):
        pred=df.iloc[idx,-1]
        gt=df.iloc[idx,-2]
        preds+=pred
        gts+=gt
    eval_predictions(preds,gts)

def eval_predictions(pred,gt):
    pred=[VOCAB.index(p) for p in tqdm(pred)]
    gt=[VOCAB.index(p) for p in tqdm(gt)]
    report = classification_report(np.array(gt),np.array(pred), labels = [1,2,3,4,5,6,7,8,9], digits=4, zero_division=0)
    print(report)    



In [8]:
def get_stat(model,df):
    df["predicted_tags"]=df.sentence.progress_apply(lambda x:run_ner_infer(" ".join(x),model))
    eval_data(df)

# Exp-1: Original

In [9]:
print("banner - original")
print("-------------------------------------")
get_stat(model_b,df)
print("scratch - original")
print("-------------------------------------")
get_stat(model_s,df)

banner - original
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1963139.87it/s]


              precision    recall  f1-score   support

           1     0.6973    0.6583    0.6772       357
           2     0.5186    0.5569    0.5370       853
           3     0.9559    0.9463    0.9511     39499
           4     0.3778    0.2982    0.3333        57
           5     0.7919    0.7874    0.7897      1223
           6     0.4578    0.4921    0.4743       573
           7     0.5152    0.6589    0.5782       516
           8     0.7879    0.7706    0.7791      2005
           9     0.7616    0.8045    0.7825      4035

    accuracy                         0.9056     49118
   macro avg     0.6515    0.6637    0.6558     49118
weighted avg     0.9084    0.9056    0.9068     49118

scratch - original
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1992608.80it/s]

              precision    recall  f1-score   support

           1     0.7128    0.5770    0.6378       357
           2     0.5456    0.4842    0.5130       853
           3     0.9468    0.9517    0.9493     39499
           4     0.3333    0.2632    0.2941        57
           5     0.7677    0.7539    0.7607      1223
           6     0.4640    0.4607    0.4623       573
           7     0.5975    0.4690    0.5255       516
           8     0.7879    0.7392    0.7627      2005
           9     0.7535    0.7968    0.7745      4035

    accuracy                         0.9030     49118
   macro avg     0.6566    0.6106    0.6311     49118
weighted avg     0.9013    0.9030    0.9019     49118



# Exp-2 Normalized

In [10]:
print("banner - normalized")
print("-------------------------------------")
get_stat(model_b,dfn)
print("scratch - normalized")
print("-------------------------------------")
get_stat(model_s,dfn)

banner - normalized
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1943985.66it/s]


              precision    recall  f1-score   support

           1     0.6469    0.6723    0.6593       357
           2     0.5344    0.5557    0.5448       853
           3     0.9564    0.9491    0.9528     39499
           4     0.3000    0.2632    0.2804        57
           5     0.7993    0.7882    0.7937      1223
           6     0.4724    0.4625    0.4674       573
           7     0.5338    0.6589    0.5898       516
           8     0.7874    0.8015    0.7944      2005
           9     0.7713    0.7965    0.7837      4035

    accuracy                         0.9082     49118
   macro avg     0.6446    0.6609    0.6518     49118
weighted avg     0.9100    0.9082    0.9090     49118

scratch - normalized
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2014588.25it/s]

              precision    recall  f1-score   support

           1     0.6993    0.5994    0.6456       357
           2     0.5726    0.5498    0.5610       853
           3     0.9574    0.9499    0.9536     39499
           4     0.4000    0.2456    0.3043        57
           5     0.7946    0.7939    0.7943      1223
           6     0.4819    0.5113    0.4962       573
           7     0.6026    0.5465    0.5732       516
           8     0.7932    0.8075    0.8003      2005
           9     0.7583    0.8327    0.7938      4035

    accuracy                         0.9109     49118
   macro avg     0.6733    0.6485    0.6580     49118
weighted avg     0.9118    0.9109    0.9111     49118



# Exp-3 Unnorm-1

In [11]:
print("banner - un-normalized-1")
print("-------------------------------------")
get_stat(model_b,df1)
print("scratch - un-normalized-1")
print("-------------------------------------")
get_stat(model_s,df1)
print("scratch - normalized-1")
print("-------------------------------------")
get_stat(model_s,df1n)

banner - un-normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2092444.66it/s]


              precision    recall  f1-score   support

           1     0.4815    0.1092    0.1781       357
           2     0.5065    0.2743    0.3559       853
           3     0.8813    0.9630    0.9203     39499
           4     0.2857    0.0351    0.0625        57
           5     0.6339    0.4955    0.5562      1223
           6     0.4096    0.1187    0.1840       573
           7     0.5140    0.3547    0.4197       516
           8     0.6603    0.3781    0.4808      2005
           9     0.6343    0.4372    0.5176      4035

    accuracy                         0.8488     49118
   macro avg     0.5563    0.3517    0.4084     49118
weighted avg     0.8263    0.8488    0.8302     49118

scratch - un-normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2018812.95it/s]


              precision    recall  f1-score   support

           1     0.5412    0.1289    0.2081       357
           2     0.5159    0.3036    0.3823       853
           3     0.8775    0.9692    0.9210     39499
           4     0.8000    0.0702    0.1290        57
           5     0.6749    0.5127    0.5827      1223
           6     0.3923    0.1431    0.2097       573
           7     0.5727    0.2442    0.3424       516
           8     0.7069    0.3392    0.4584      2005
           9     0.7075    0.4520    0.5516      4035

    accuracy                         0.8537     49118
   macro avg     0.6432    0.3514    0.4206     49118
weighted avg     0.8338    0.8537    0.8336     49118

scratch - normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1987878.96it/s]


              precision    recall  f1-score   support

           1     0.6163    0.1485    0.2393       357
           2     0.5191    0.3341    0.4066       853
           3     0.8843    0.9683    0.9244     39499
           4     0.6000    0.1053    0.1791        57
           5     0.6779    0.5593    0.6129      1223
           6     0.4903    0.2216    0.3053       573
           7     0.5579    0.2616    0.3562       516
           8     0.7201    0.3696    0.4885      2005
           9     0.7159    0.4766    0.5722      4035

    accuracy                         0.8592     49118
   macro avg     0.6424    0.3828    0.4538     49118
weighted avg     0.8420    0.8592    0.8419     49118



# Exp-4 Unnorm-2

In [12]:
print("banner - un-normalized-2")
print("-------------------------------------")
get_stat(model_b,df2)
print("scratch - un-normalized-2")
print("-------------------------------------")
get_stat(model_s,df2)
print("scratch - normalized-2")
print("-------------------------------------")
get_stat(model_s,df2n)

banner - un-normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2054057.69it/s]


              precision    recall  f1-score   support

           1     0.4211    0.0672    0.1159       357
           2     0.4986    0.2098    0.2954       853
           3     0.8646    0.9677    0.9133     39499
           4     0.3750    0.0526    0.0923        57
           5     0.6245    0.3876    0.4783      1223
           6     0.4196    0.0820    0.1372       573
           7     0.5243    0.3140    0.3927       516
           8     0.6254    0.2873    0.3937      2005
           9     0.6018    0.3554    0.4469      4035

    accuracy                         0.8372     49118
   macro avg     0.5505    0.3026    0.3629     49118
weighted avg     0.8084    0.8372    0.8109     49118

scratch - un-normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2028713.18it/s]


              precision    recall  f1-score   support

           1     0.5526    0.0588    0.1063       357
           2     0.4973    0.2181    0.3032       853
           3     0.8577    0.9776    0.9137     39499
           4     0.8000    0.0702    0.1290        57
           5     0.6536    0.3827    0.4827      1223
           6     0.4806    0.1082    0.1766       573
           7     0.6074    0.1919    0.2916       516
           8     0.7120    0.2429    0.3622      2005
           9     0.6948    0.3425    0.4588      4035

    accuracy                         0.8413     49118
   macro avg     0.6507    0.2881    0.3583     49118
weighted avg     0.8177    0.8413    0.8106     49118

scratch - normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2052809.18it/s]

              precision    recall  f1-score   support

           1     0.5000    0.0616    0.1097       357
           2     0.5117    0.2309    0.3183       853
           3     0.8594    0.9778    0.9148     39499
           4     0.8571    0.1053    0.1875        57
           5     0.6739    0.4039    0.5051      1223
           6     0.5409    0.1501    0.2350       573
           7     0.6164    0.1899    0.2904       516
           8     0.7181    0.2554    0.3767      2005
           9     0.7027    0.3445    0.4623      4035

    accuracy                         0.8434     49118
   macro avg     0.6645    0.3022    0.3778     49118
weighted avg     0.8212    0.8434    0.8139     49118



# Exp-5 Unnorm 5

In [13]:
print("banner - un-normalized-5")
print("-------------------------------------")
get_stat(model_b,df5)
print("scratch - un-normalized-5")
print("-------------------------------------")
get_stat(model_s,df5)
print("scratch - normalized-5")
print("-------------------------------------")
get_stat(model_s,df5n)

banner - un-normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2124684.92it/s]


              precision    recall  f1-score   support

           1     0.5385    0.0196    0.0378       357
           2     0.4208    0.0996    0.1611       853
           3     0.8304    0.9836    0.9005     39499
           4     0.0000    0.0000    0.0000        57
           5     0.6167    0.1447    0.2344      1223
           6     0.3962    0.0366    0.0671       573
           7     0.4242    0.1085    0.1728       516
           8     0.5856    0.1382    0.2236      2005
           9     0.5653    0.1641    0.2543      4035

    accuracy                         0.8171     49118
   macro avg     0.4864    0.1883    0.2280     49118
weighted avg     0.7738    0.8171    0.7657     49118

scratch - un-normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1961457.69it/s]


              precision    recall  f1-score   support

           1     0.4545    0.0140    0.0272       357
           2     0.5746    0.0903    0.1560       853
           3     0.8204    0.9937    0.8987     39499
           4     0.0000    0.0000    0.0000        57
           5     0.6626    0.1316    0.2196      1223
           6     0.5000    0.0244    0.0466       573
           7     0.7059    0.0465    0.0873       516
           8     0.7772    0.0713    0.1307      2005
           9     0.6875    0.1090    0.1882      4035

    accuracy                         0.8167     49118
   macro avg     0.5758    0.1645    0.1949     49118
weighted avg     0.7909    0.8167    0.7534     49118

scratch - normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1991337.61it/s]

              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       357
           2     0.5678    0.0785    0.1380       853
           3     0.8191    0.9948    0.8984     39499
           4     0.0000    0.0000    0.0000        57
           5     0.7265    0.1325    0.2241      1223
           6     0.4333    0.0227    0.0431       573
           7     0.6800    0.0329    0.0628       516
           8     0.7833    0.0703    0.1291      2005
           9     0.7258    0.1024    0.1794      4035

    accuracy                         0.8165     49118
   macro avg     0.5262    0.1593    0.1861     49118
weighted avg     0.7905    0.8165    0.7516     49118

